In [27]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [28]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

for i in range(len(files)):
    if 'E_no' in files[i]:
        if '__' not in files[i]:
            if 'all' not in files[i]:
                filenames.append(files[i])
                print(i, files[i])

3 Ag_100_u2_RPBE_no001_CO_n1_d9.traj
7 Ag_100_u2_RPBE_no002_CO_n1_d9.traj
11 Ag_100_u2_RPBE_no003_CO_n1_d9.traj
17 Ag_110_u2_RPBE_no001_CO_n1_d9.traj
21 Ag_110_u2_RPBE_no002_CO_n1_d9.traj
25 Ag_110_u2_RPBE_no003_CO_n1_d9.traj
29 Ag_110_u2_RPBE_no004_CO_n1_d9.traj
35 Ag_111_u2_RPBE_no001_CO_n1_d9.traj
39 Ag_111_u2_RPBE_no002_CO_n1_d9.traj
43 Ag_111_u2_RPBE_no003_CO_n1_d9.traj
47 Ag_111_u2_RPBE_no004_CO_n1_d9.traj
53 Ag_211_u2_RPBE_no001_CO_n1_d9.traj
57 Ag_211_u2_RPBE_no002_CO_n1_d9.traj
61 Ag_211_u2_RPBE_no003_CO_n1_d9.traj
65 Ag_211_u2_RPBE_no004_CO_n1_d9.traj
69 Ag_211_u2_RPBE_no005_CO_n1_d9.traj
73 Ag_211_u2_RPBE_no006_CO_n1_d9.traj
77 Ag_211_u2_RPBE_no007_CO_n1_d9.traj
81 Ag_211_u2_RPBE_no008_CO_n1_d9.traj
85 Ag_211_u2_RPBE_no009_CO_n1_d9.traj
89 Ag_211_u2_RPBE_no010_CO_n1_d9.traj
95 Al_100_u2_RPBE_no001_CO_n1_d9.traj
99 Al_100_u2_RPBE_no002_CO_n1_d9.traj
103 Al_100_u2_RPBE_no003_CO_n1_d9.traj
109 Al_110_u2_RPBE_no001_CO_n1_d9.traj
113 Al_110_u2_RPBE_no002_CO_n1_d9.traj
117 Al_110_

In [29]:
# ### DELETE ###
# for file in filenames:
#     try:
#         A = make_database(file)
#         A.delete_from_database()
#     except:
#         print(file, 'Some error occured.')

In [32]:
### ADD and UPDATE ENERGY ###
for file in filenames:
    try:
        A = make_database(file)
        A.add_to_database()
        A.update_Energy()
        A.update_adsorbates_correlation()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
--------------------------------

-----------------------------------------------------------
Au_111_u2_RPBE_no002_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Au_111_u2_RPBE_no003_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Au_111_u2_RPBE_no004_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Au_211_u2_RPBE_no001_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
--------------------------------

-----------------------------------------------------------
Ni_111_u2_RPBE_no001_CO_n1_d8.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ni_111_u2_RPBE_no002_CO_n1_d8.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ni_111_u2_RPBE_no003_CO_n1_d8.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ni_111_u2_RPBE_no004_CO_n1_d8.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
--------------------------------

Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Pt_211_u2_RPBE_no008_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Pt_211_u2_RPBE_no009_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Pt_211_u2_RPBE_no010_CO_n1_d9.traj
Already in database
Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Rh_100_u2_RPBE_no001_CO_n1_d8.traj
Already in database
Updating ene

In [31]:
for file in filenames:
    try:
        A = make_database(file)
        data = A.check_database()
        print(data['name'], data['minimum_distance'], data['ads_dist2'], data['ads_dist3'])
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Ag_100_u2_RPBE_no001_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Ag_100_u2_RPBE_no002_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Ag_100_u2_RPBE_no003_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
Ag_110_u2_RPBE_no001_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no002_CO_n1_d9.traj
Ag_110_u2_RPBE_no002_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no003_CO_n1_d9.traj
Ag_110_u2_RPBE_no003_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no004_CO_n1_d9.traj
Ag_110_u2_RPBE_no004_CO_n1_d9.traj 

-----------------------------------------------------------
Cu_211_u2_RPBE_no003_CO_n1_d8.traj
Cu_211_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no004_CO_n1_d8.traj
Cu_211_u2_RPBE_no004_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no005_CO_n1_d8.traj
Cu_211_u2_RPBE_no005_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no006_CO_n1_d8.traj
Cu_211_u2_RPBE_no006_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no007_CO_n1_d8.traj
Cu_211_u2_RPBE_no007_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no008_CO_n1_d8.traj
Cu_211_u2_RPBE_no008_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no009_CO_n1_d8.traj
Cu_211_u2_RPBE_no009_CO_n1_d8.traj 

Rh_110_u2_RPBE_no002_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_110_u2_RPBE_no003_CO_n1_d8.traj
Rh_110_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_110_u2_RPBE_no004_CO_n1_d8.traj
Rh_110_u2_RPBE_no004_CO_n1_d8.traj None None None
-----------------------------------------------------------
Rh_111_u2_RPBE_no001_CO_n1_d8.traj
Rh_111_u2_RPBE_no001_CO_n1_d8.traj None None None


In [21]:
# for file in filenames:
#     try:
#         A = make_database(file)
#         print(A.check_database()['minimum_distance'])
#     except:
#         print(file, 'Some error occured.')

In [22]:
file = 'Cu_100_u2_RPBE_no006_CO_n4_d3.traj'
A = make_database(file)
A.check_database()

No file named Cu_100_u2_RPBE_no006_CO_n4_d3.traj in init
No file named Cu_100_u2_RPBE_no006_CO_n4_d3.traj in database
-----------------------------------------------------------
Cu_100_u2_RPBE_no006_CO_n4_d3.traj
Not in database.


In [47]:
query('Cu_110_u2_RPBE_no003_CO_n3_d3.traj','local')

Atoms(symbols='Cu16COCOCO', pbc=[True, True, False], cell=[7.354476072672618, 5.200399903061017, 23.900299927295762], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))